# Задание №5

### Содержание задания


Создать три датасета с «качественными данными», на которых можно обучить простую модель линейной регрессии, например: На одном из этих датасетов обучить модель линейной регрессии. Создать датасет с шумом в данных Провести тестирование работы модели на разных датасетах с использованием Pytest, анализируя качество предсказания, обнаружить проблему на датасете с шумами.

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import ipytest


### создаем датасет 

In [27]:
np.random.seed(73) 
num_samples = 1000  # количество примеров

# Генерация случайных признаков
data = {
    'feature_1': np.random.rand(num_samples),
    'feature_2': np.random.rand(num_samples),
    'feature_3': np.random.rand(num_samples),
    'target': np.random.rand(num_samples) * 15  # Целевая переменная, например, цены
}

df = pd.DataFrame(data)
df.head()

,feature_1,feature_2,feature_3,target
0,0.642690,0.242081,0.164459,0.472554
1,0.538621,0.406316,0.189583,14.768626
2,0.511898,0.190863,0.088655,10.303683
3,0.610257,0.577323,0.406597,14.014651
4,0.492989,0.843785,0.618470,11.128562


## Создание датасета с шумом

In [37]:
noise_level = 0.2  # уровень шума
f_noise = np.random.randn(num_samples,3) * noise_level

# Добавление шума к данным
noisy_features_data = data.copy()
for i in range(1, 3):
    noisy_features_data[f'feature_{i}'] += f_noise[:, i-1]


df_noisy = pd.DataFrame(noisy_features_data)

df_noisy.head()


,feature_1,feature_2,feature_3,target
0,0.959743,1.221995,0.188260,0.472554
1,0.456938,0.639301,-0.758808,14.768626
2,-1.485018,0.027188,0.368721,10.303683
3,1.159325,1.895795,1.153481,14.014651
4,-0.001923,0.732979,0.215295,11.128562


In [38]:
# Генерация шума для целевой переменной
target_noise = np.random.randn(num_samples) * noise_level * 8  # Больший шум для целевой переменной

# Добавление шума к целевой переменной
noisy_target_data = data.copy()
noisy_target_data['target'] += target_noise

# Создание DataFrame с шумной целевой переменной
df_noisy_target = pd.DataFrame(noisy_target_data)
df_noisy_target.head()

,feature_1,feature_2,feature_3,target
0,0.959743,1.221995,0.188260,2.690017
1,0.456938,0.639301,-0.758808,14.211018
2,-1.485018,0.027188,0.368721,10.331531
3,1.159325,1.895795,1.153481,13.621214
4,-0.001923,0.732979,0.215295,11.173890


### Модель линейной регрессии

In [39]:
# Разделение данных на обучающую и тестовую выборку
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=73)

In [40]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [41]:
# Оценка модели на тестовой выборке
predict = model.predict(X_test)
mse = mean_squared_error(y_test, predict)
print("MSE:", mse)

MSE: 16.861597193382508


## Написание тестов с использованием pytest

In [42]:
ipytest.autoconfig()

In [43]:
def test_model():
    X_noisy = df_noisy.drop('target', axis=1)
    y_noisy = df_noisy['target']
    _, X_test_noisy_features, _, y_test_noisy_features = train_test_split(X_noisy, y_noisy, test_size=0.3, random_state=3)
    predict = model.predict(X_test_noisy_features)
    mse = mean_squared_error(y_test_noisy_features, predict)
    print("MSE :", mse)
    assert mse > 10 

In [44]:
# проверяем пройдет или нет
ipytest.run('-v')

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.2.0, pluggy-1.5.0
rootdir: /home/olga/lab5
plugins: time-machine-2.13.0, anyio-4.2.0, hydra-core-1.3.2
collected 2 items

t_e9a62741d89f4acfaba49742dfc844dc.py ..                                                     [100%]

======================================== 2 passed in 0.01s =========================================


<ExitCode.OK: 0>

тест пройден

In [51]:
def test_model_max():
    X_noisy = df_noisy.drop('target', axis=1)
    y_noisy = df_noisy['target']
    _, X_test_noisy_features, _, y_test_noisy_features = train_test_split(X_noisy, y_noisy, test_size=0.3, random_state=3)
    predict = model.predict(X_test_noisy_features)
    mse = mean_squared_error(y_test_noisy_features, predict)
    print("MSE :", mse)
    assert mse < 10

In [52]:
ipytest.run('-v')

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.2.0, pluggy-1.5.0
rootdir: /home/olga/lab5
plugins: time-machine-2.13.0, anyio-4.2.0, hydra-core-1.3.2
collected 2 items

t_e9a62741d89f4acfaba49742dfc844dc.py .F                                                     [100%]

============================================= FAILURES =============================================
__________________________________________ test_model_max __________________________________________

    def test_model_max():
        X_noisy = df_noisy.drop('target', axis=1)
        y_noisy = df_noisy['target']
        _, X_test_noisy_features, _, y_test_noisy_features = train_test_split(X_noisy, y_noisy, test_size=0.3, random_state=3)
        predict = model.predict(X_test_noisy_features)
        mse = mean_squared_error(y_test_noisy_features, predict)
        print("MSE :", mse)
>       assert mse < 10
E       assert 17.

<ExitCode.TESTS_FAILED: 1>

тест не пройден. 
(MSE на зашумленных данных = 17.5)